In [1]:
import pandas as pd

In [2]:
df =  pd.read_csv('customer_segmentation_data.csv')

In [3]:
df

,id,age,gender,income,spending_score,membership_years,purchase_frequency,preferred_category,last_purchase_amount
0,1,38,Female,99342,90,3,24,Groceries,113.53
1,2,21,Female,78852,60,2,42,Sports,41.93
2,3,60,Female,126573,30,2,28,Clothing,424.36
3,4,40,Other,47099,74,9,5,Home & Garden,991.93
4,5,65,Female,140621,21,3,25,Electronics,347.08
...,...,...,...,...,...,...,...,...,...
995,996,57,Male,112170,57,6,1,Clothing,313.64
996,997,23,Other,65337,76,10,23,Groceries,632.83
997,998,23,Male,113097,40,5,42,Sports,75.09
998,999,22,Female,113695,63,7,44,Electronics,505.16


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    1000 non-null   int64  
 1   age                   1000 non-null   int64  
 2   gender                1000 non-null   object 
 3   income                1000 non-null   int64  
 4   spending_score        1000 non-null   int64  
 5   membership_years      1000 non-null   int64  
 6   purchase_frequency    1000 non-null   int64  
 7   preferred_category    1000 non-null   object 
 8   last_purchase_amount  1000 non-null   float64
dtypes: float64(1), int64(6), object(2)
memory usage: 70.4+ KB


In [5]:
df['income'] = df['income'].astype(float).round(2).apply(lambda x: f'{x:.2f}')

In [6]:
df.describe()

,id,age,spending_score,membership_years,purchase_frequency,last_purchase_amount
count,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000
mean,500.500000,43.783000,50.685000,5.46900,26.596000,492.348670
std,288.819436,15.042213,28.955175,2.85573,14.243654,295.744253
min,1.000000,18.000000,1.000000,1.00000,1.000000,10.400000
25%,250.750000,30.000000,26.000000,3.00000,15.000000,218.762500
50%,500.500000,45.000000,50.000000,5.00000,27.000000,491.595000
75%,750.250000,57.000000,76.000000,8.00000,39.000000,747.170000
max,1000.000000,69.000000,100.000000,10.00000,50.000000,999.740000
